# Convolutional Neural Network

* CIFAR-10 dataset

## 1. Settings

### 1-1 Import Required Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 1-2 Set Hyperparameters

In [2]:
batch_size=256
lr = 0.0002
num_epoch = 100

## 2.Data Generation

### 2-1 Download Data

In [3]:
train = dset.CIFAR10("./", train=True, transform = transforms.ToTensor(),target_transform=None, download=True)
test = dset.CIFAR10("./", train=False, transform = transforms.ToTensor(),target_transform=None, download=True)

Files already downloaded and verified
Files already downloaded and verified


### 2-2 Check Dataset

In [4]:
train.__getitem__(0)[0].size()

torch.Size([3, 32, 32])

In [5]:
train[0]

(tensor([[[ 0.2314,  0.1686,  0.1961,  ...,  0.6196,  0.5961,  0.5804],
          [ 0.0627,  0.0000,  0.0706,  ...,  0.4824,  0.4667,  0.4784],
          [ 0.0980,  0.0627,  0.1922,  ...,  0.4627,  0.4706,  0.4275],
          ...,
          [ 0.8157,  0.7882,  0.7765,  ...,  0.6275,  0.2196,  0.2078],
          [ 0.7059,  0.6784,  0.7294,  ...,  0.7216,  0.3804,  0.3255],
          [ 0.6941,  0.6588,  0.7020,  ...,  0.8471,  0.5922,  0.4824]],
 
         [[ 0.2431,  0.1804,  0.1882,  ...,  0.5176,  0.4902,  0.4863],
          [ 0.0784,  0.0000,  0.0314,  ...,  0.3451,  0.3255,  0.3412],
          [ 0.0941,  0.0275,  0.1059,  ...,  0.3294,  0.3294,  0.2863],
          ...,
          [ 0.6667,  0.6000,  0.6314,  ...,  0.5216,  0.1216,  0.1333],
          [ 0.5451,  0.4824,  0.5647,  ...,  0.5804,  0.2431,  0.2078],
          [ 0.5647,  0.5059,  0.5569,  ...,  0.7216,  0.4627,  0.3608]],
 
         [[ 0.2471,  0.1765,  0.1686,  ...,  0.4235,  0.4000,  0.4039],
          [ 0.0784,  0.0000,

In [6]:
train.__len__()

50000

### 2-3 Set DataLoader

In [7]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

## 3. Model & Optimizer

### 3-1 CNN Model

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), #(32,16,16)
            
            nn.Conv2d(32,64,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64,128,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),   #(128, 8,8)
            
            nn.Conv2d(128,256,3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256,256,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),   #(256, 4,4)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(256*4*4, 200),
            nn.ReLU(),
            nn.Linear(200,10)
        )
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        
        return out
    
model = CNN()

### 3-2 Loss func & Optimizer

In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

### 3-3 Load Model

In [10]:
try:
    model = torch.load('./CIFAR10_model.pkl')
    print("model restored")
except:
    print("model not restored")

model restored


## 4. Train

In [11]:
for epoch in range(num_epoch):
    for i, data in enumerate(train_loader):
        images, labels = data
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        y_pred = model(images)
        loss=loss_func(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print(epoch, i, loss)

0 0 tensor(1.3608)
0 100 tensor(1.3584)
1 0 tensor(1.2876)
1 100 tensor(1.2864)
2 0 tensor(1.3759)
2 100 tensor(1.4371)
3 0 tensor(1.3937)
3 100 tensor(1.3517)
4 0 tensor(1.5467)
4 100 tensor(1.2504)
5 0 tensor(1.5045)
5 100 tensor(1.3973)
6 0 tensor(1.4053)
6 100 tensor(1.3797)
7 0 tensor(1.4832)
7 100 tensor(1.4435)
8 0 tensor(1.4051)
8 100 tensor(1.5168)


Process Process-17:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", lin

KeyboardInterrupt: 

In [11]:
param_list = list(model.parameters())
print(param_list)

[tensor([[[[ 0.1946, -0.0362, -0.1258],
          [ 0.0965, -0.1355,  0.0869],
          [-0.0786,  0.0869,  0.1567]],

         [[ 0.1664, -0.1692,  0.0176],
          [-0.0851, -0.1885, -0.1907],
          [ 0.1222, -0.0861,  0.0795]],

         [[-0.1403,  0.1790,  0.1670],
          [-0.1753, -0.0384, -0.0808],
          [-0.0399,  0.0418,  0.0523]]],


        [[[ 0.0474,  0.1246,  0.0220],
          [-0.1328, -0.1793,  0.0247],
          [-0.0561,  0.0765, -0.1056]],

         [[ 0.1102, -0.1290,  0.1470],
          [ 0.1276, -0.0903,  0.1833],
          [ 0.1638, -0.1350,  0.1162]],

         [[ 0.1651,  0.0185, -0.0699],
          [-0.0703,  0.1870, -0.1743],
          [-0.0635,  0.0203, -0.1347]]],


        [[[ 0.1218,  0.0010,  0.1826],
          [ 0.1228,  0.1472,  0.0611],
          [ 0.1172, -0.0985, -0.1902]],

         [[ 0.1486,  0.1759, -0.0338],
          [-0.1266, -0.0016, -0.1642],
          [ 0.0483, -0.1120, -0.1284]],

         [[ 0.0625, -0.0532, -0.0028],
    

In [12]:
len(param_list)

16

In [16]:
param_list[0].size()

torch.Size([16, 3, 3, 3])

## 5. Test

In [23]:
correct = 0
total = 0

for images, labels in test_loader:
    images = Variable(images, volatile=True)
    labels = Variable(labels)
    
    output = model(images)
    _,output_index = torch.max(output, 1)
    
    total += labels.size(0)
    correct += (output_index ==labels).sum().float()
    
print(f"Accuracy of Test Data: {100*correct/total : .3f}")
    
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Accuracy of Test Data:  47.877
